In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.18.0 requires google-genai<2.0.0,>=1.45.0, but you have google-genai 1.7.0 which is incompatible.
google-cloud-aiplatform 1.125.0 requires google-genai<2.0.0,>=1.37.0, but you have google-genai 1.7.0 which is incompatible.


In [2]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display, clear_output

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [3]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [4]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

#### Testing the model

In [5]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

Imagine you have a really, really smart puppy that can learn tricks. That puppy is like AI!

Instead of learning tricks, AI learns from lots and lots of information. Think of it like showing the puppy a million pictures of cats so it knows exactly what a cat looks like.

AI can do things like:

*   **Recognize things:** Like knowing that's a dog in a picture, or understanding your voice when you ask it to play music.
*   **Learn patterns:** Like figuring out what your favorite TV show is based on what you watch.
*   **Make predictions:** Like guessing what word you're going to say next when you're typing a message.
*   **Solve problems:** Like finding the shortest route on a map.

It's not magic, it's just really good at figuring things out based on all the information it's been shown. AI is getting smarter and smarter all the time! Sometimes it can even seem like it's thinking, but really it's just using what it's learned. It needs people to show it what to learn, and to help it learn

In [6]:
Markdown(response.text)

Imagine you have a really, really smart puppy that can learn tricks. That puppy is like AI!

Instead of learning tricks, AI learns from lots and lots of information. Think of it like showing the puppy a million pictures of cats so it knows exactly what a cat looks like.

AI can do things like:

*   **Recognize things:** Like knowing that's a dog in a picture, or understanding your voice when you ask it to play music.
*   **Learn patterns:** Like figuring out what your favorite TV show is based on what you watch.
*   **Make predictions:** Like guessing what word you're going to say next when you're typing a message.
*   **Solve problems:** Like finding the shortest route on a map.

It's not magic, it's just really good at figuring things out based on all the information it's been shown. AI is getting smarter and smarter all the time! Sometimes it can even seem like it's thinking, but really it's just using what it's learned. It needs people to show it what to learn, and to help it learn correctly.


#### Start a chat

In [7]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('Hello! My name is Hemant.')
print(response.text)

Hello Hemant! It's nice to meet you. How can I help you today?



In [8]:
response = chat.send_message('Can you tell me something interesting about black holes?')
Markdown(response.text)

Okay, here's something interesting about black holes that might blow your mind:

**While black holes are known for sucking in everything, they also play a crucial role in the formation of new stars!**

Here's the breakdown:

*   **Accretion Disks and Jets:** Black holes are often surrounded by accretion disks – swirling masses of gas and dust pulled towards the black hole. As this material spirals inward, it heats up and releases tremendous amounts of energy. Some of this energy is expelled outwards in powerful jets of plasma that shoot out from the black hole's poles at near light speed.

*   **Triggering Star Formation:** These jets, though incredibly powerful, can also compress surrounding clouds of gas and dust. This compression can provide the "push" needed for these clouds to overcome their internal pressure and collapse under their own gravity, initiating the process of star formation.

*   **Controlling Star Formation:** While they can trigger it, black holes also regulate star formation. If the jets are *too* powerful, they can actually disrupt gas clouds and prevent star formation. This means that black holes act like cosmic regulators, preventing regions from becoming *too* dense with stars.

So, while black holes are notorious for their destructive nature, they also play a vital, and perhaps counterintuitive, role in the ongoing cycle of creation and destruction in the universe. They're not just cosmic vacuums; they're complex engines that influence the birth of new stars.

Is there anything else you'd like to know about black holes or any other topic?


In [9]:
response = chat.send_message('Do you remember what my name is?')
print(response.text)

Yes, your name is Hemant.



#### Available models

In [10]:
for model in client.models.list():
  print(model.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

#### Additional info about the model

In [11]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent',
                       'countTokens',
                       'createCachedContent',
                       'batchGenerateContent'],
 'tuned_model_info': {},
 'version': '2.0'}


#### Explore generation parameters

In [12]:
short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write 1000 word essay on modern corporate life and stress that comes with it.'
)

Markdown(response.text)

## The Gilded Cage: Stress and the Modern Corporate Life

The modern corporate landscape, a complex ecosystem of ambition, innovation, and cutthroat competition, has become synonymous with a particular type of stress. It's a stress that permeates not just the workplace but bleeds into the personal lives of its participants, shaping their habits, relationships, and overall well-being. While corporate life offers undeniable rewards – financial security, professional development, and a sense of purpose – it often comes at a steep cost: a relentless pressure to perform, adapt, and succeed in a world that is constantly evolving. This essay will explore the various facets of stress within the modern corporate world, examining the contributing factors, the individual and societal consequences, and ultimately, contemplating the need for a more sustainable and human-centric approach.

One of the most significant contributors to corporate stress is the **demanding work culture**. Driven by the relentless pursuit of productivity and profitability, companies often expect employees to be "always on,"

In [13]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='Write short poem on modern corporate life and stress that comes with it.'
)

Markdown(response.text)

Sunrise paints the glass so high,
Another day to strive and lie.
A thousand emails, urgent, stark,
Another meeting in the dark.

The screen glows, a hypnotic gleam,
Distorting truth, a hollow dream.
Deadlines loom, a crushing weight,
Another soul begins to break.

Success defined in profit's name,
Erosion of a human frame.
The coffee's cold, the spirit frayed,
Another cog, forever swayed.


#### Playing with temperature

In [14]:
temp_config = types.GenerateContentConfig(temperature=2.0)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Cerulean

#############
Emerald

#############
Emerald

#############
Teal

#############
Teal

#############


In [15]:
temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Azure

#############
Azure

#############
Azure

#############
Azure

#############
Azure

#############


In [16]:
temp_config = types.GenerateContentConfig(temperature=0.2)

for _ in range(5):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=temp_config,
        contents='generate a color (respond with just one word)'
    )

    if response.text:
        print(response.text)
        print('#############')

Azure

#############
Azure

#############
Azure

#############
Azure

#############
Azure

#############


In [17]:
model_config = types.GenerateContentConfig(
    temperature=2.0,
    top_p=0.95,
)

prompt = 'Write a story about a cat who quits his corporate job and retires to a peaceful life in the mountains'

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=prompt
)

Markdown(response.text)

Bartholomew Buttons III, a ginger tabby of impeccable pedigree (if he did say so himself), sat at his desk. Or, more accurately, perched upon it. He glared at the spreadsheet projected on the wall – "Q3 Revenue Projections: Premium Tuna Treats." The fluorescent lights hummed a relentless, soul-crushing buzz. He smoothed down his meticulously groomed fur, a gesture that had become a tic in the last few months.

For five years, Bartholomew had climbed the corporate ladder at "Whiskers & Co.," purveyors of the finest feline cuisine. He had clawed his way from junior taste tester to Vice President of Tuna Innovation, a title he found vaguely embarrassing. But the constant pressure, the never-ending deadlines, the office politics disguised as innocent head-butts… it was all too much.

He swiveled his tail irritably. Suddenly, he saw himself – reflected in the sleek, stainless steel desk lamp – a corporate automaton. A cat devoid of joy. The realization hit him like a poorly aimed hairball.

“That’s it!” he announced to the startled programmer a few desks over. He didn't bother waiting for a response. He was already unplugging his ergonomic, heat-controlled cat bed and grabbing his favorite laser pointer.

That afternoon, Bartholomew walked into Mr. Fluffernutter’s corner office – a mahogany mausoleum smelling faintly of catnip and despair – and slapped his letter of resignation (written in elegant paw script on parchment) onto the desk.

Mr. Fluffernutter, a portly Persian with a permanent air of dissatisfaction, barely glanced up. “Bartholomew, are you sure? This company has big plans for you. We were thinking of making you… Executive Vice President of Salmon Strategic Planning!”

Bartholomew shook his head. “I’m done, Mr. Fluffernutter. Done with the meetings, the memos, the constant chasing of the red dot… metaphorical and otherwise. I'm retiring.”

He paused, channeling his inner dramatic feline. "I'm going to find myself. In the mountains.”

He spun on his heel and strode out, the squeak of his polished paws on the expensive rug echoing his freedom.

The mountains. He had always dreamed of them. Of pine-scented air, rustling leaves, and the absence of PowerPoint presentations. He sold his luxury penthouse apartment overlooking the tuna processing plant, packed a backpack with essentials – salmon jerky, a self-warming blanket, a very specific brand of catnip tea – and hopped on the next northbound train.

His mountain cabin was everything he’d hoped for. Cozy, rustic, and blessed with an abundance of sunbeams. He spent his days napping in hammocks strung between towering pines, chasing butterflies in meadows brimming with wildflowers, and perfecting the art of the purr.

He discovered hidden springs and drank from them, the water colder and fresher than any artificially flavored hydration concoction Whiskers & Co. had ever attempted. He learned to identify the tracks of local wildlife - the delicate prints of deer, the telltale signs of playful squirrels. He even made a friend, a grumpy badger named Bernard who grudgingly shared his berry patch.

At night, Bartholomew would sit on his porch, a mug of catnip tea steaming in his paws, and gaze at the star-dusted sky. The city’s incessant hum was replaced by the chirping of crickets and the hooting of owls. He felt a peace he had never known at Whiskers & Co.

One evening, Bernard shuffled up to the porch. “Hear you used to be… something fancy back in the city,” he grumbled, sniffing the air appreciatively. "Tuna dude or something?"

Bartholomew nodded. "Vice President of Tuna Innovation."

Bernard snorted. "Innovation, eh? Try innovatin' this," he said, and tossed a plump, juicy blackberry onto Bartholomew's lap. "That's innovation, Button. Been around longer than your fancy tuna."

Bartholomew looked at the blackberry, its sweetness a promise on his tongue. He chuckled. Bernard, despite his gruff exterior, was right. He popped the berry into his mouth and closed his eyes, savoring the simple, natural flavor.

He had come to the mountains searching for peace and meaning. He found them not in spreadsheets and strategy meetings, but in the taste of wild berries, the warmth of the sun on his fur, and the quiet companionship of a grumpy old badger. He had finally, truly, retired. And for the first time in a long time, Bartholomew Buttons III felt content. His corporate days were over. His cat nap, on the other hand, was just beginning.


#### Prompting

##### Zero shot prompt

In [18]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5
)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt
)

print(response.text)

POSITIVE



### Enum mode

In [19]:
import enum

class Sentiment(enum.Enum):
    POSITIVE= 'pos'
    NEGETIVE= 'neg'
    NEUTRAL= 'neu'

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt
)

print(response.text)

pos


In [20]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.POSITIVE
<enum 'Sentiment'>


#### Example workflow

In [21]:
from google.genai.types import Part

class CustomerIntent(enum.Enum):
    CANCEL_ORDER = 'cancel_order'
    CHECK_STATUS = 'check_status'
    REQUEST_REFUND = 'request_refund'
    GENERAL_QUERY = 'general_query'
    PRODUCT_INFO = 'product_info'

SYSTEM_PROMPT = (
    "You are an expert intent classifier for a retail company. "
    "Your task is to analyze the user's message and determine the single, primary intent. "
    "You MUST only respond with one of the provided enum values."
)

def classify_intent(user_message):
    contents = [
        # Put the system prompt as a user-role content (server accepts "user")
        types.Content(
            role="user",
            parts=[Part.from_text(text=SYSTEM_PROMPT)]
        ),
        # Actual user message
        types.Content(
            role="user",
            parts=[Part.from_text(text=user_message)]
        ),
    ]


    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=contents, 
            config=types.GenerateContentConfig(
                response_mime_type="text/x.enum",
                response_schema=CustomerIntent
            )
        )
        return response.text.strip()
    
    except Exception as e:
        return f"API Error: {e}"

In [22]:
message_1 = "I need to get my money back because the item arrived broken."
intent_1 = classify_intent(message_1)
print(intent_1)

message_2 = "Can you tell me where my package is right now? I ordered it last week."
intent_2 = classify_intent(message_2)

message_3 = "Does the new X-Pro 500 come in red, and what are the main features?"
intent_3 = classify_intent(message_3)

request_refund


In [23]:
def trigger_workflow(intent: str, message: str):
    """Simulates a backend routing workflow based on the classified intent."""
    
    print(f"\nUser Message: '{message}'")
    print(f"Classified Intent: {intent}")
    
    match intent:
        case CustomerIntent.REQUEST_REFUND.value:
            print("Routing: Initiating Refund Request Workflow.")
        case CustomerIntent.CHECK_STATUS.value:
            print("Routing: Querying Shipping Database for Order Status.")
        case CustomerIntent.CANCEL_ORDER.value:
            print("Routing: Sending Order Cancellation Confirmation.")
        case CustomerIntent.PRODUCT_INFO.value:
            print("Routing: Connecting to Product Knowledge Base.")
        case _:
            # This should not happen if the enum enforcement works
            print("ERROR: Received an invalid intent. Escalating to human review.")


# Run the workflow simulation
trigger_workflow(intent_1, message_1)
trigger_workflow(intent_2, message_2)
trigger_workflow(intent_3, message_3)


User Message: 'I need to get my money back because the item arrived broken.'
Classified Intent: request_refund
Routing: Initiating Refund Request Workflow.

User Message: 'Can you tell me where my package is right now? I ordered it last week.'
Classified Intent: check_status
Routing: Querying Shipping Database for Order Status.

User Message: 'Does the new X-Pro 500 come in red, and what are the main features?'
Classified Intent: product_info
Routing: Connecting to Product Knowledge Base.


#### Few shot prompt

In [24]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = "Give me a large with half margherita & half burrata salad"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])


print(response.text)

```json
{
  "size": "large",
  "type": "half and half",
  "ingredients_1": ["tomato sauce", "mozzarella", "basil"],
  "ingredients_2": ["burrata", "salad"]
}
```


#### JSON output

In [25]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="Can I have a large margherita pizza?")

print(response.text)

{
  "size": "large",
  "ingredients": ["tomato sauce", "mozzarella cheese", "basil"],
  "type": "margherita"
}


#### Exploring CoT

In [26]:
prompt = """When I was 8 years old, my sister was half my age. Now, I
am 34 years old. How old is my sister? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

30



In [27]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

48



In [28]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

Here's how to solve the problem step-by-step:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.

2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3. **Determine your partner's current age:** Since the age difference remains constant, your partner is always 8 years older than you. Now that you are 20, your partner is 20 + 8 = 28 years old.

**Answer:** Your partner is currently 28 years old.


#### Exploring ReAct

In [29]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

In [30]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)


react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
I need to find the transformers NLP paper and then identify the youngest author listed on the paper.

Action 1
<search>transformers NLP paper</search>



In [31]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
The paper is Attention Is All You Need. The authors listed are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin. I need to find the youngest author of these authors. Since I cannot directly look up the age of the authors, I will start with the first author and search the author.

Action 2
<search>Ashish Vaswani</search>



#### Thinking mode (using experimental thinking model)

The experiemental Gemini Flash 2.0 "Thinking" model has been trained to generate the "thinking process" the model goes through as part of its response. As a result, the Flash Thinking model is capable of stronger reasoning capabilities in its responses.

Using a "thinking mode" model can provide you with high-quality responses without needing specialised prompting like the previous approaches. One reason this technique is effective is that you induce the model to generate relevant information ("brainstorming", or "thoughts") that is then used as part of the context in which the final response is generated.

In [32]:
import io

response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='Who was the youngest author listed on the transformers NLP paper?',
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    # Display the response as it is streamed
    print(chunk.text, end='')

# And then render the finished response as formatted markdown.
clear_output()
Markdown(buf.getvalue())

The youngest author listed on the "Attention Is All You Need" paper (which introduced the Transformer architecture, often referred to as the Transformers NLP paper) was **Aidan N. Gomez**.

He was 22 years old when the paper was published in 2017.

#### Code generation

In [35]:
code_prompt = """
Write a Python function to calculate the factorial of a number. No explanation, provide only the code.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1.0,
        top_p=1.0,
        max_output_tokens=1000
    ),
    contents=code_prompt
)

Markdown(response.text)

```python
def factorial(n):
  """
  Calculate the factorial of a number.

  Args:
    n: An integer.

  Returns:
    The factorial of n.
  """
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)
```


#### Code execution

In [38]:
config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())]
)

code_exec_prompt = """
Generate the first 14 odd prime numbers, then calculate their sum.
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="Okay, I understand. First, I need to generate the first 14 odd prime numbers. Remember that a prime number is a number greater than 1 that has only two divisors: 1 and itself. The first few primes are 2, 3, 5, 7, 11, etc. Since I need *odd* prime numbers, I will exclude 2 from the list. Then, I'll calculate their sum.\n\n"), Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=ExecutableCode(code="primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]\nsum_of_primes = sum(primes)\nprint(f'{primes=}')\nprint(f'{sum_of_primes=}')\n", language=<Language.PYTHON: 'PYTHON'>), file_data=None, function_call=None, function_response=None, inline_data=None, text=None), Part(video_metadata=None, thought=None, code_execut

In [52]:
for part in response.candidates[0].content.parts:
    pprint(part.to_json_dict())
    print("-----")

{'text': 'Okay, I understand. First, I need to generate the first 14 odd prime '
         'numbers. Remember that a prime number is a number greater than 1 '
         'that has only two divisors: 1 and itself. The first few primes are '
         '2, 3, 5, 7, 11, etc. Since I need *odd* prime numbers, I will '
         "exclude 2 from the list. Then, I'll calculate their sum.\n"
         '\n'}
-----
{'executable_code': {'code': 'primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, '
                             '37, 41, 43, 47]\n'
                             'sum_of_primes = sum(primes)\n'
                             "print(f'{primes=}')\n"
                             "print(f'{sum_of_primes=}')\n",
                     'language': 'PYTHON'}}
-----
{'code_execution_result': {'outcome': 'OUTCOME_OK',
                           'output': 'primes=[3, 5, 7, 11, 13, 17, 19, 23, 29, '
                                     '31, 37, 41, 43, 47]\n'
                                     'sum_of_primes

In [53]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## Status {part.code_execution_result.outcome}'))

        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

Okay, I understand. First, I need to generate the first 14 odd prime numbers. Remember that a prime number is a number greater than 1 that has only two divisors: 1 and itself. The first few primes are 2, 3, 5, 7, 11, etc. Since I need *odd* prime numbers, I will exclude 2 from the list. Then, I'll calculate their sum.



```python
primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes = sum(primes)
print(f'{primes=}')
print(f'{sum_of_primes=}')

```

```
primes=[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes=326

```

The first 14 odd prime numbers are 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, and 47. Their sum is 326.


#### Explain code using Gemini

In [54]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

This file is a **Bash script that enhances your command-line prompt to display Git repository status information**.  In other words, it makes your prompt aware of Git.

Here's a breakdown:

*   **What it is:** It's a collection of shell functions and configuration designed to be sourced (included) in your `.bashrc` (or equivalent) file. When sourced, it modifies how your command prompt looks.

*   **What it does:**  The core function, `setGitPrompt` (and related helper functions), figures out if the current directory is part of a Git repository. If it is, it extracts information like the current branch, the state of the working directory (modified, staged, untracked files), and the relationship with remote branches (ahead, behind).  It then formats this information and adds it to your prompt using color-coded symbols.

*   **Why use it?**
    *   **Git Awareness:**  Quickly see the status of your Git repository without having to run `git status` manually.
    *   **Improved Workflow:** Makes it easier to manage your Git repositories by keeping you informed about changes and branch information at a glance.
    *   **Customization:** Allows you to customize the colors, symbols, and information displayed in the prompt to suit your preferences.
    *   **Virtual Environment Awareness**: Can be configured to display your currently active virtual environment.

In short, this script provides a visually informative and convenient way to keep track of your Git repositories directly in your command-line prompt.  Instead of a plain prompt like `user@host:~$`, you might see something like `user@host:~/my-repo (main↓2↑1*)` indicating you are on the `main` branch, 2 commits behind and 1 commit ahead of the remote, with some local changes.
